# Spatial Databases II

## Spatial Queries

> **Spatial queries are queries in a spatial database** that can be answered on the **basis of geometric information only, i.e., the spatial position and extent of the objects involved**.

### Containment Query (st_contains)

The function **st_contains(geometry A,geometry B)** returns true if geometry A completely contains geometry B (**Note: make sure the SRID's are the same**)

![Contains_explanation](images/containment_detail.png) 

Some real world examples include

![PointInPolygon](images/PointInPolygon.png) 

![Covid Cases](images/Covid_Cases.png) 

Lets look at an example. In this example we want to find how many schools are with in the Zone with locationid 47. So we are going to query the tables taxi_zones and nyc_schools. First let us check the CRS information of both the tables

```sql
select st_srid(geom) from nyc_schools limit 1
select st_srid(geom) from taxi_zones limit 1
```

Seems like both have CRS 26918...Great.. So safe to proceed. We only want Zone with locationid 47

```sql
select n.gid from nyc_schools n,taxi_zones t where t.locationid=47 and st_contains(t.geom,n.geom)
```

Which will give you all the unique schoold ids. Now if we want to get the count

```sql
select count(n.gid) as total_count from nyc_schools n,taxi_zones t where t.locationid=47 and st_contains(t.geom,n.geom)
```


Another example to find out the total schools in each taxi zones

```sql
select t.locationid,count(n.gid) as total_count from nyc_schools n,taxi_zones t where st_contains(t.geom,n.geom) group by t.locationid
```

What if you want to also show taxizones without any schools. 



### Within Query (st_within)

With in query is very similar to contains query and **st_within(geometryA,geometryB)** checks whether geometryA is completely within geometryB. Let us look at a simple example. 

Find all schools that have a shooting incident with in 50 meters of its location. Here we have to use nypd_shooting and nyc_schools. First check the SRIDs and then proceed

```sql
select sc.gid,sc.schoolname from nypd_shooting s,nyc_schools sc where st_within(s.geom,st_buffer(sc.geom,50))
```

Now find out the total count of shooting incidents near each school. 

### Intersection Query (st_intersects)

The function **st_intersects(geometry A,geometry B)** returns true if geometry A and geometry B intersect or touch at atleast a single point

![LineIntersects](images/intersect.png) 

A real world example would be to identify the houses that fall with in a hazard zone. We would not only want the houses that are with in the hazard zone but also the houses that has some portion of it inside the hazard zone

![Hazard Zones](images/hazard_zones.png) 

Let us try out an example. 

Find all the residential streets that pass through Soho neighborhood in Manhattan. We have to use nyc_neighborhoods table and nyc_streets table

```sql
select st.name from nyc_streets st, nyc_neighborhoods n where st_intersects(st.geom,n.geom) and n.boroname='Manhattan' and n.name = 'Soho'```

### With in a Distance Queries (st_dwithin)

With in distance queries are used to find out geometrical objects that are with in a specific distance of a particular geometrical object.

![Withindistance_example](images/withindistance.png)

Let us look at some concrete examples

We want to find out schools that are within 100 meters of an nyc subway station. 

```sql
select sc.schoolname,s.gid,s.name from nyc_schools sc,nyc_subway_stations s where st_dwithin(sc.geom,s.geom,100)
```